# Research Process 


In [2]:
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [8]:
#load data
df=pd.read_csv("breast-cancer-wisconsin-data/data.csv")
#drop irelevent columns for the classification
df = df.drop(columns=['Unnamed: 32', 'id'])
# rearange the data for X - featuers and Y leabels 
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [9]:
#
#split the data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#hendeling imblance data 
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
#the class weight is done only on the train data to impact the learning process and to evaluete beter the model proformence
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: np.float64(0.7954545454545454), 1: np.float64(1.3461538461538463)}

In [26]:
classes=np.unique(y_train)
classes ,class_weight_dict.values()


(array(['B', 'M'], dtype=object),
 dict_values([np.float64(0.7954545454545454), np.float64(1.3461538461538463)]))

as prdicted the minorty class 'M' get higer weight of 1.346